## Load and Clean Course Metadata

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/courses.csv')

# Get the column titles

column_titles = df.columns.tolist()
print("Column Titles:", column_titles)

df_clean = df.dropna(subset=['Unique Disp names', 'one fourth'])

sampling_list = dict(zip(df_clean['Unique Disp names'], df_clean['one fourth']))

print(sampling_list)

In [ ]:
#Random sampling per discipline
selected_rows=[]
for key, value in sampling_list.items():
  filtered_df = df[df['Discipline Name'] == key]

  sampled_rows = filtered_df.sample(n=int(value), random_state=42)
  sampled_rows = sampled_rows.iloc[:, :-4]  
  selected_rows.append(sampled_rows)

final_df = pd.concat(selected_rows)

final_df = final_df.iloc[:, 1:]
final_df.insert(0, 'S.No', range(1, len(final_df) + 1))
final_df.to_csv('final_selected_rows.csv', index=False)

print(final_df)

## Environment Setup for Chrome & Selenium to Download videos from NPTEL based on course wise

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'dpkg' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'dpkg' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/91/08/10cff8463b3510b78f9e3dcef6b37c542b06d71ed1240a8940ba0c75d3bc/selenium-4.26.1-py3-none-any.whl.metadata
  Obtaining dependency information for chromedriver_autoinstaller from https://files.pythonhosted.org/packages/a5/b5/36f0b0add145c371b5282e881a687601899f2d27fae5d0595bc02026b67c/chromedriver_autoinstaller-0.6.4-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/3c/83/ec3196c360afffbc5b342ead48d1eb7393dd74fa70bca75d33905a86f211/trio-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions~=4.9 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda

In [1]:

# Import necessary modules
import chromedriver_autoinstaller
from selenium import webdriver

# Automatically install chromedriver matching the installed version of Chrome
chromedriver_autoinstaller.install()


'c:\\Python312\\Lib\\site-packages\\chromedriver_autoinstaller\\130\\chromedriver.exe'

In [2]:
import time
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sys
import chromedriver_autoinstaller

sys.path.insert(0,':\\Users\\ninav\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\130\\chromedriver.exe')

In [3]:
os.getcwd()

'c:\\Users\\ninav\\EVD'

In [4]:
import requests


def download_video(url, filename):
    # Start downloading the video
    print(f"Starting download for {filename}...")

    # Streaming the file to handle large files
    response = requests.get(url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        content_type = response.headers.get('Content-Type')

        if content_type != "video/mp4":
          print(f"Skipping {filename}, not an MP4 file. Content-Type: {content_type}")
          return

        with open(filename, 'wb') as file:
            # Writing the content in chunks to avoid memory overload
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(os.path.getsize(filename),int(response.headers.get('Content-Length', 0)))
        print(f"Done: {filename}")
    else:
        print(f"Failed to download {filename}, HTTP Status Code: {response.status_code}")

In [10]:
import ffmpeg
import subprocess

def get_video_info(path):
    probe = ffmpeg.probe(path)

    for stream in probe["streams"]:
        if stream["codec_type"] == "video":
            return stream


In [ ]:
apt-get install -y ffmpeg


In [6]:
def to_timestamp(ts):
    h, m, s = int(ts // 3600), int((ts // 60) % 60), int(ts % 60)
    return f'{h:02}:{m:02}:{s:02}'



In [8]:
import os
import json
import subprocess

l = ["https://nptel.ac.in/courses/106106131"]

for item in l:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless') # this is must
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chromedriver_autoinstaller.install()

    driver = webdriver.Chrome(options=chrome_options)

    #set the web responsive screeen
    driver.set_window_size(1920, 1080)
    driver.get(item) # Website used for scraping
    time.sleep(7)

    #Displaying the title of the website in this case I had used GFG's Website
    print(driver.title,"\n")
    element = driver.find_elements(By.CSS_SELECTOR, 'span.tab')[1]


    element.click()

    # Step 3: Retrieve the updated HTML content

    updated_html = driver.page_source
    #print(updated_html)
    element = driver.find_elements(By.CSS_SELECTOR, 'span.tab')[1]

    # Option 1: Print the text content of the element
    element_xpath = '/html/body/app-root/app-course-details/main/section/app-course-detail-ui/div/div[3]/app-course-downloads/div/div[2]/div[2]/h3[1]'
    parent_element = driver.find_element(By.XPATH, '//h3[contains(text(), "Videos")]/ancestor::div[2]')
    span_html_content = parent_element.get_attribute('outerHTML')
    print("HTML Content of the selected span:")
 
    # Find all <a> tags inside the located element
    links = parent_element.find_elements(By.TAG_NAME, 'a')
    print(len(links))
    # Print all the links (href attribute)
    print("\nExtracted Links:")
    for link in links[0:1]:
        url = link.get_attribute('href')

        print(url)
        url_parts = url.split('/')
        course_code = url_parts[-3]  # Extract course code part (e.g., 106101208)
        video_name = url_parts[-1]
        file_name = f"{video_name}.json"
        video_info = get_video_info("/content/mod04lec19.mp4")

        write_metadata(f"{course_code}\\{video_name}", to_timestamp(video_info["duration"]), f"{video_info['width']}*{video_info['height']}")
        print(f"New rows have been appended to '{csv_file}'.")

        # write the duration of the video 
        # Write dictionary to JSON file
        with open(file_name, 'w') as json_file:
            json.dump(video_info, json_file, indent=4)  # Use 'indent' for pretty formatting

        print(f"Dictionary has been written to {file_name}")


        file_path =f"C:\\Users\\ninav\\EVD\\courses\\{course_code}\\{video_name}"
        os.makedirs(os.path.dirname(file_path), exist_ok=True)

        
        download_video(url,file_path)



    # Close the browser

    driver.quit()

NPTEL 

HTML Content of the selected span:
56

Extracted Links:
https://media.dev.nptel.ac.in/content/mp4/106/106/106106131/MP4/mod01lec01.mp4


AttributeError: module 'ffmpeg' has no attribute 'probe'

In [ ]:
import csv

# File name
csv_file = "video_metadata.csv"

# Column headers
columns = ["Video ID", "Duration (s)", "Extracted Frames", "Frame Resolution"]

# Open the file in write mode to write the column headers
with open(csv_file, mode="w", newline="") as file:
    # Initialize CSV writer
    writer = csv.writer(file)
    
    # Write only the headers (column names)
    writer.writerow(columns)

print(f"Column names have been written to '{csv_file}'.")


Column names have been written to 'video_metadata.csv'.


In [7]:
import csv

def write_metadata(video_id, duration, resolution):
# File name
    csv_file = "video_metadata.csv"

    # Additional video metadata to append
    new_video_data = [
        {"Video ID": "vid004", "Duration (s)": 240, "Frame Resolution": "1920x1080"},
        {"Video ID": "vid005", "Duration (s)": 150, "Frame Resolution": "1280x720"},
    ]

    # Open the CSV file in append mode and write the new rows
    with open(csv_file, mode="a", newline="") as file:
        # Initialize CSV writer
        writer = csv.DictWriter(file, fieldnames=["Video ID", "Duration (s)", "Extracted Frames", "Frame Resolution"])
        
        # Append new rows
        for video in new_video_data:
            writer.writerow({
                "Video ID": video_id,
                "Duration (s)": duration,
                "Extracted Frames": "",  # Leave empty
                "Frame Resolution": resolution
            })

